In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import sys
import os
sys.path.append( os.getcwd()+'/diffusion_maps')

import numpy as np
import diffusion_maps.diffusionmap as dm
import scipy.sparse as sps

In [ ]:
import pandas as pd
#trajLoaded=np.asarray(pd.read_csv('trajectory-friction_constant_1-input_columns_1_2-inverse_temperature_1-max_steps_10000-sampler_StochasticMomentumLangevin-step_width_1e-2.csv', sep=','))
#trajLoaded=np.asarray(pd.read_csv('trajectory-friction_constant_1-input_columns_1_2-inverse_temperature_1e5-max_steps_10000-sampler_StochasticMomentumLangevin-step_width_1e-2.csv', sep=','))
#trajLoaded=np.asarray(pd.read_csv('trajectory-friction_constant_1e5-input_columns_1_2-inverse_temperature_1-max_steps_10000-sampler_StochasticMomentumLangevin-step_width_1e-2.csv', sep=','))
#trajLoaded=np.asarray(pd.read_csv('trajectory-friction_constant_1e5-input_columns_1_2-inverse_temperature_1e5-max_steps_10000-sampler_StochasticMomentumLangevin-step_width_1e-2.csv', sep=','))

trajLoaded=np.asarray(pd.read_csv('trajectory-friction_constant_1-input_columns_1_2-inverse_temperature_1e5-max_steps_100000-sampler_StochasticMomentumLangevin-step_width_1e-2.csv', sep=','))


modNr=10
start=500

traj=trajLoaded[:,2:]
loss = trajLoaded[:, 1]
print(traj.shape)
traj=traj[start::modNr,:]
loss = trajLoaded[start::modNr, 1]


print("Variance is ", traj.var(axis=0))

In [ ]:
#print(traj[:10,0])
conv=np.zeros(traj.shape)

for i in range(1,traj.shape[0]):
    for d in range(traj.shape[1]):
        
        conv[i,d]=np.mean(traj[:i,d])

[plt.scatter(range(len(traj)), conv[:,i]) for i in range(traj.shape[1])]
plt.show()

print(conv[-1,:])

In [ ]:
import importlib
importlib.reload(dm)
epsilon=0.1

#landmarks, V1 = sampler.dimension_reduction(traj, epsilon, numberOfLandmarks, smpl.model, smpl.T, method=Method)
kernelDiff=dm.compute_kernel(traj, epsilon)
P=dm.compute_P(kernelDiff, traj)
q=kernelDiff.sum(axis=1)

#P, target_distribution, qEmp = compute_unweighted_P( X, epsilon, sampler)
lambdas, V = sps.linalg.eigsh(P, k=4)#, which='LM' )
ix = lambdas.argsort()[::-1]
lambdas=lambdas[ix]
X_se= V[:,ix]

V1=X_se[:,1]    
V2=X_se[:,2]
V3=X_se[:,3]

print(ix)
print(lambdas[ix])

In [ ]:
plt.scatter(range(len(lambdas)),lambdas)
plt.ylabel('lambda')
plt.show()

plt.scatter(traj[:,0], traj[:,1], c= loss)
plt.xlabel('w1')
plt.ylabel('w2')
plt.colorbar()
plt.show()

plt.scatter(V1, V2, c= loss)
plt.xlabel('V1')
plt.ylabel('V2')
plt.show()

plt.scatter(V1, V3)
plt.xlabel('V1')
plt.ylabel('V3')
plt.show()

plt.scatter(traj[:,0], np.asarray(q))
plt.scatter(traj[:,1], np.asarray(q))
plt.scatter(traj[:,2], np.asarray(q))
plt.xlabel('w')
plt.ylabel('q')
plt.show()

plt.scatter(traj[:,0], traj[:,1], c=np.asarray(q))
plt.xlabel('w_1')
plt.ylabel('w_2')
plt.show()

plt.scatter(traj[:,0], traj[:,1], c=V1)
plt.xlabel('w_1')
plt.ylabel('w_2')
plt.title('W1 and W2 colored by V1')
plt.show()

# plt.scatter(traj[:,0], V1, c='b', s=10)
# plt.xlabel('w1')
# plt.ylabel('V')
# plt.show()

# plt.scatter(traj[:,1], V1, c='b', s=10)
# plt.xlabel('w2')
# plt.ylabel('V')
# plt.show()


# plt.scatter(traj[:,2], V1, c='b', s=10)
# plt.xlabel('w3')
# plt.ylabel('V')
# plt.show()

# plt.scatter(traj[:,3], V1, c='b', s=10)
# plt.xlabel('w4')
# plt.ylabel('V')
# plt.show()

In [ ]:
def pdist2(x,y):
    v=np.sqrt(((x-y)**2).sum())
    return v

def get_landmarks(data, K, q, V1, E):

    m = float(q.size)
    #q=np.array(q)

    delta = 100/m*(max(V1)-min(V1))
    deltaMax=2*delta
    levels = np.linspace(min(V1),max(V1),num=K)

    lb = 1

    landmarks=np.zeros(K)
    emptyLevelSet=0

    for k in range(K-1, -1, -1):


            levelsetLength=0

            # we want to identify idices in V1 which are delta close to the levels
            #---o----o----o----o----o---
            #  *o** *o*  *o*  *o*   o
            # if there are no indeces in the delta distance, increase the delta distance

            while levelsetLength==0:

                levelset = np.where(np.abs(V1 - levels[k]) < delta)
                levelset=levelset[0]
                levelsetLength=len(levelset)
 
                delta=delta*1.001

                if delta>deltaMax:
                    levelset=range(0,len(V1))
  
            data_level = data[levelset,:]

            if k==K-1:
                
                idx = np.argmin(E[levelset]/m )
                landmarks[k]= levelset[idx]

            else:
                
                idx = np.argmin(E[levelset]/m )
                qtmp=E[levelset]/m

                # compute the distance to the last landmark
                dist_to_last=np.zeros(data_level.shape[0])
                for i in range(0,data_level.shape[0]):
                    dist_to_last[int(i)] = pdist2(data[int(landmarks[k+1]),:], data_level[int(i)])
                dtmp=np.array(dist_to_last.reshape(qtmp.shape))

                v=qtmp  - lb*dtmp

                idx = np.argmax(v);

                landmarks[k]= levelset[idx]


    return landmarks.astype(int)

In [ ]:
numberOfLandmarks=20

# compute landmarks
landmarks=get_landmarks(traj, numberOfLandmarks, q, V1,  loss)

plt.scatter(traj[:,0], traj[:,1], c=V1) 
plt.scatter(traj[landmarks,0], traj[landmarks,1], c='r') 
plt.plot(traj[landmarks,0], traj[landmarks,1], color='k') 
plt.xlabel('w0')
plt.ylabel('w1')

plt.show()


plt.scatter(traj[:,0], traj[:,2], c=V1) 
plt.scatter(traj[landmarks,0], traj[landmarks,2], c='r') 
plt.plot(traj[landmarks,0], traj[landmarks,2], color='k') 
plt.xlabel('w0')
plt.ylabel('w2')
plt.show()

plt.scatter(traj[:,1], traj[:,2], c=V1) 
plt.scatter(traj[landmarks,1], traj[landmarks,2], c='r') 
plt.plot(traj[landmarks,1], traj[landmarks,2], color='k') 
plt.xlabel('w1')
plt.ylabel('w2')
plt.show()

In [ ]:
#compute levelsets
levelsets, dummy = dm.get_levelsets(traj, numberOfLandmarks, q, V1)

K=len(levelsets)
freeEnergy=np.zeros(K)
h=np.zeros(K)

for k in range(0,K):
    h[k] = len(levelsets[k])

for k in range(0,K):
    freeEnergy[k] = -np.log(h[k]/sum(h))

In [ ]:
plt.plot(loss[landmarks], freeEnergy, '*-b')
plt.xlabel('loss')
plt.ylabel('F')
plt.show()

plt.plot(traj[landmarks, 0], freeEnergy, '*-b')
plt.xlabel('w0')
plt.ylabel('F')
plt.show()
plt.plot(traj[landmarks, 1], freeEnergy, '*-g')
plt.xlabel('w1')
plt.ylabel('F')
plt.show()
plt.plot(traj[landmarks, 2], freeEnergy, '*-k')
plt.xlabel('w2')
plt.ylabel('F')
plt.show()